In [1]:
import requests
import matplotlib.pyplot as plt
import json
import pandas as pd

In [2]:
def url_to_df(url):
    endpoint = url
    response = requests.get(endpoint)
    response
    if response.status_code == 200:
        F1 = response.json()
        F1_df = pd.DataFrame(F1)
    else: print(response.status_code)
    return F1_df

In [3]:
def agg_to_df(aggdf):
    x = aggdf.session_key.drop_duplicates().reset_index(drop = True)
    x = sessions.loc[sessions.session_key.isin(x)].reset_index(drop=True)
    return x

In [4]:
sessions = url_to_df('https://api.openf1.org/v1/sessions')
sessions = sessions[['session_key','location','session_name','year']]

In [5]:
sessions = sessions[sessions.year != 2025]
sessions = sessions[sessions.session_name == 'Race']
sessions = sessions.reset_index(drop = True)
sessions.head()

,session_key,location,session_name,year
0,7953,Sakhir,Race,2023
1,7779,Jeddah,Race,2023
2,7787,Melbourne,Race,2023
3,9070,Baku,Race,2023
4,9078,Miami,Race,2023


In [6]:
session_list = sessions.session_key.to_list()

In [7]:
weather = url_to_df('https://api.openf1.org/v1/weather')
weather = weather[weather.session_key.isin(session_list)]

In [8]:
weather['date'] = pd.to_datetime(weather['date'],format='ISO8601')
weather['date'] = weather['date'].dt.strftime('%Y-%m-%d %H:%M')
weather['air_temperature'] = (weather.air_temperature	* (9/5)) + 32
weather['track_temperature'] = (weather.track_temperature	* (9/5)) + 32
weather['wind_speed'] = (weather.wind_speed * 2.237)
weather = weather.rename(columns = {'date':'date_start'})
weather = weather.reset_index(drop = True)
weather

,air_temperature,humidity,pressure,rainfall,track_temperature,wind_direction,wind_speed,date_start,session_key,meeting_key
0,85.64,19.0,1016.5,0,95.18,176,2.6844,2023-03-05 14:01,7953,1141
1,85.46,19.0,1016.5,0,95.00,182,2.6844,2023-03-05 14:02,7953,1141
2,85.46,19.0,1016.5,0,94.82,156,2.4607,2023-03-05 14:03,7953,1141
3,85.28,19.0,1016.5,0,94.82,201,1.7896,2023-03-05 14:04,7953,1141
4,85.28,19.0,1016.5,0,94.64,219,1.7896,2023-03-05 14:05,7953,1141
...,...,...,...,...,...,...,...,...,...,...
7392,78.62,60.0,1018.0,0,84.74,122,5.8162,2024-12-08 14:30,9662,1252
7393,78.62,60.0,1018.0,0,84.92,84,5.3688,2024-12-08 14:31,9662,1252
7394,78.44,60.0,1018.0,0,84.92,34,2.6844,2024-12-08 14:32,9662,1252
7395,78.44,60.0,1018.0,0,84.38,116,2.6844,2024-12-08 14:33,9662,1252


In [9]:
drivers = url_to_df('https://api.openf1.org/v1/drivers')
drivers = drivers[['session_key','driver_number','full_name','team_name']]
drivers = drivers[drivers.session_key.isin(session_list)]

In [10]:
driver_list = [4,16,23]
drivers = drivers[drivers.driver_number.isin(driver_list)]
drivers

,session_key,driver_number,full_name,team_name
135,7953,4,Lando NORRIS,None
139,7953,16,Charles LECLERC,None
144,7953,23,Alexander ALBON,None
235,7779,4,Lando NORRIS,None
239,7779,16,Charles LECLERC,None
...,...,...,...,...
4528,9655,16,Charles LECLERC,Ferrari
4532,9655,23,Alexander ALBON,Williams
4624,9662,4,Lando NORRIS,McLaren
4628,9662,16,Charles LECLERC,Ferrari


In [11]:
lando_laps = url_to_df('https://api.openf1.org/v1/laps?driver_number=4')
lando_laps = lando_laps[['session_key','date_start','driver_number','i1_speed','i2_speed','st_speed','lap_duration','duration_sector_1','duration_sector_2','duration_sector_3','lap_number']]
lando_laps['date_start'] = pd.to_datetime(lando_laps['date_start'],format='ISO8601')
lando_laps['date_start'] = lando_laps['date_start'].dt.strftime('%Y-%m-%d %H:%M:%S')
lando_laps = lando_laps[lando_laps.session_key.isin(session_list)]
lando_laps

,session_key,date_start,driver_number,i1_speed,i2_speed,st_speed,lap_duration,duration_sector_1,duration_sector_2,duration_sector_3,lap_number
219,7953,NaN,4,229.0,236.0,263.0,NaN,NaN,44.789,24.561,1
220,7953,2023-03-05 15:05:24,4,231.0,238.0,291.0,100.433,31.867,43.830,24.736,2
221,7953,2023-03-05 15:07:04,4,232.0,255.0,300.0,99.777,31.627,43.448,24.702,3
222,7953,2023-03-05 15:08:44,4,227.0,224.0,286.0,102.267,32.947,44.387,24.933,4
223,7953,2023-03-05 15:10:26,4,225.0,248.0,315.0,100.664,32.022,43.875,24.767,5
...,...,...,...,...,...,...,...,...,...,...,...
6432,9662,2024-12-08 14:22:47,4,NaN,306.0,312.0,88.124,17.591,37.872,32.661,53
6433,9662,2024-12-08 14:24:15,4,NaN,300.0,309.0,87.647,17.662,37.925,32.060,54
6434,9662,2024-12-08 14:25:43,4,NaN,301.0,312.0,87.698,17.599,37.985,32.114,55
6435,9662,2024-12-08 14:27:10,4,NaN,302.0,313.0,87.929,17.586,37.999,32.344,56


In [12]:
albon_laps = url_to_df('https://api.openf1.org/v1/laps?driver_number=23')
albon_laps = albon_laps[['session_key','date_start','driver_number','i1_speed','i2_speed','st_speed','lap_duration','duration_sector_1','duration_sector_2','duration_sector_3','lap_number']]
albon_laps['date_start'] = pd.to_datetime(albon_laps['date_start'],format='ISO8601')
albon_laps['date_start'] = albon_laps['date_start'].dt.strftime('%Y-%m-%d %H:%M')
albon_laps = albon_laps[albon_laps.session_key.isin(session_list)]

In [13]:
leclerc_laps = url_to_df('https://api.openf1.org/v1/laps?driver_number=16')
leclerc_laps = leclerc_laps[['session_key','date_start','driver_number','i1_speed','i2_speed','st_speed','lap_duration','duration_sector_1','duration_sector_2','duration_sector_3','lap_number']]
leclerc_laps['date_start'] = pd.to_datetime(leclerc_laps['date_start'],format='ISO8601')
leclerc_laps['date_start'] = leclerc_laps['date_start'].dt.strftime('%Y-%m-%d %H:%M')
leclerc_laps = leclerc_laps[leclerc_laps.session_key.isin(session_list)]

In [14]:
sessions.to_csv('../data/session_api.csv')
weather.to_csv('../data/weather_api.csv')
drivers.to_csv('../data/drivers_api.csv')
lando_laps.to_csv('../data/laps_lando_api.csv')
albon_laps.to_csv('../data/laps_albon_api.csv')
leclerc_laps.to_csv('../data/laps_leclerc_api.csv')